In [ ]:
import csv
import plotly.offline as py
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from networkx.algorithms.community import k_clique_communities
from networkx.algorithms import community

## Load PPI edges

In [ ]:
ppi_folder = "PPI_databases/"

In [ ]:
ppi_edges_dict = {}

In [ ]:
# Append iREF data to edge dict
import os

for file_name in os.listdir(ppi_folder):
    with open(ppi_folder + file_name,"r") as file:
        print(file_name)
        print(len(ppi_edges_dict))
        if not file_name.endswith(".sig"):
            continue
        for line in file.read().split("\n"):
            data = line.split()
            if len(data) != 13:
                continue
            if data[0] in ppi_edges_dict:
                ppi_edges_dict[data[0]].add(data[5])
            else:
                ppi_edges_dict[data[0]] = set([data[5]])

In [ ]:
# compile a list of all the edges as tuples
ppi_edges_list = []
for gene_a,val in ppi_edges_dict.items():
    for gene_b in val:
        ppi_edges_list.append((gene_a,gene_b))
    
print(len(ppi_edges_list))

In [ ]:
# write the resulting list to a file
with open("ppi_edges_list.csv","w") as f_out:
    writer = csv.writer(f_out, delimiter=',', lineterminator='\n')
    writer.writerows(ppi_edges_list)



In [ ]:
df = pd.read_csv("ppi_edges_list.csv", header=None)
df.head()

## Load gene-gene coexpression edges

In [ ]:
df = pd.read_feather("human_correlation_archs4.f")
print(df.shape)
df.head()

In [ ]:
# create array of gene names
names = df.columns.tolist()

# initialize dict
top_corr = {}

top_n = 5

for i in range(len(names)):
    col = df[names[i]]
    col.index = names
    col = col.sort_values(ascending=False)
    top = [ x for x in col.index.values[0:top_n + 1] if x != names[i] ]
    top_corr[names[i]] = top

In [ ]:
df_top = pd.DataFrame(top_corr)
df_top.to_csv(f"top_{top_n}_correlation.csv", encoding='utf-8', index=False)
df_top.head()

In [ ]:
gene_edges_dict = df_top.to_dict('list')